In [1]:
# Code adapted from Machine Learning Engineering (Cornell Tech 2025)
import torch
import numpy as np
import random

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [2]:
from google.colab import drive
import sys
import os

# --- 1. Mount Drive ---
drive.mount('/content/drive')

# --- 2. Define Paths ---
# Path to the source code (loaders.py) - REMAINS ON DRIVE
DRIVE_CODE_PATH = '/content/drive/MyDrive/GoogleColab/dataforsptransformer/'

# Path to the zipped data file on Drive
ZIP_SOURCE_PATH = os.path.join(DRIVE_CODE_PATH, 'crc_markers.zip')

# Local disk folder where the FAST images will be unzipped
FAST_DATA_PATH = '/content/fast_data/'

# --- 3. Unzip Data (Performance Fix) ---
if not os.path.exists(FAST_DATA_PATH):
    print(f"🚀 Unzipping data from Drive to fast local disk: {FAST_DATA_PATH}")
    !mkdir -p "$FAST_DATA_PATH"
    # The -q flag silences the output. -d sets the destination directory.
    !unzip -q "$ZIP_SOURCE_PATH" -d "$FAST_DATA_PATH"

    print("✅ Data transfer complete. Starting new batch load test.")
else:
    print("Fast data directory already exists.")


# --- 4. Set Final Variables ---
# PROJECT_DIR for the rest of your notebook now points to the FAST images
PROJECT_DIR = FAST_DATA_PATH

# Add the Drive path for Python to find 'loaders.py' and other modules
if DRIVE_CODE_PATH not in sys.path:
    sys.path.append(DRIVE_CODE_PATH)
    print(f"✅ Added {DRIVE_CODE_PATH} to Python system path.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fast data directory already exists.
✅ Added /content/drive/MyDrive/GoogleColab/dataforsptransformer/ to Python system path.


In [3]:
from google.colab import files

# This will open a 'Choose Files' button in your output cell
# Choose train_helpers.py
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [4]:
!python3 data_cleaning.py

In [7]:
import pandas as pd

df = pd.read_csv("/content/fast_data/CRC_clusters_neighborhoods_markers_cleaned.csv")
df.head()

,CellID,ClusterID,EventID,File Name,Region,TMA_AB,TMA_12,Index in File,groups,patients,...,CD8+ICOS+,CD8+Ki67+,CD8+PD-1+,Treg-ICOS+,Treg-Ki67+,Treg-PD-1+,neighborhood number final,neighborhood name,GraphID,KNN
0,0,10668,0,reg001_A,reg001,A,1,0,1,1,...,0,0,0,0,0,0,9.0,Granulocyte enriched,reg001_A 1 reg001 1,"[79342, 215276, 86261, 19386, 85435]"
1,1,10668,4,reg001_A,reg001,A,1,4,1,1,...,0,0,0,0,0,0,4.0,Macrophage enriched,reg001_A 1 reg001 1,"[209687, 147136, 231069, 61712, 50718]"
2,2,10668,5,reg001_A,reg001,A,1,5,1,1,...,0,0,0,0,0,0,3.0,Immune-infiltrated stroma,reg001_A 1 reg001 1,"[116916, 169947, 195850, 32707, 214032]"
3,3,10668,6,reg001_A,reg001,A,1,6,1,1,...,0,0,0,0,0,0,3.0,Immune-infiltrated stroma,reg001_A 1 reg001 1,"[7427, 215280, 163828, 223298, 93734]"
4,4,10668,30,reg001_A,reg001,A,1,30,1,1,...,0,0,0,0,0,0,4.0,Macrophage enriched,reg001_A 1 reg001 1,"[158193, 153016, 27846, 87820, 97886]"


In [8]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00


In [12]:
df.columns

Index(['CellID', 'ClusterID', 'EventID', 'File Name', 'Region', 'TMA_AB',
       'TMA_12', 'Index in File', 'groups', 'patients', 'spots',
       'CD44 - stroma:Cyc_2_ch_2', 'FOXP3 - regulatory T cells:Cyc_2_ch_3',
       'CD8 - cytotoxic T cells:Cyc_3_ch_2',
       'p53 - tumor suppressor:Cyc_3_ch_3',
       'GATA3 - Th2 helper T cells:Cyc_3_ch_4',
       'CD45 - hematopoietic cells:Cyc_4_ch_2', 'T-bet - Th1 cells:Cyc_4_ch_3',
       'beta-catenin - Wnt signaling:Cyc_4_ch_4', 'HLA-DR - MHC-II:Cyc_5_ch_2',
       'PD-L1 - checkpoint:Cyc_5_ch_3', 'Ki67 - proliferation:Cyc_5_ch_4',
       'CD45RA - naive T cells:Cyc_6_ch_2', 'CD4 - T helper cells:Cyc_6_ch_3',
       'CD21 - DCs:Cyc_6_ch_4', 'MUC-1 - epithelia:Cyc_7_ch_2',
       'CD30 - costimulator:Cyc_7_ch_3', 'CD2 - T cells:Cyc_7_ch_4',
       'Vimentin - cytoplasm:Cyc_8_ch_2', 'CD20 - B cells:Cyc_8_ch_3',
       'LAG-3 - checkpoint:Cyc_8_ch_4', 'Na-K-ATPase - membranes:Cyc_9_ch_2',
       'CD5 - T cells:Cyc_9_ch_3', 'IDO-1 - metaboli

In [ ]:
icols = ['CD44 - stroma:Cyc_2_ch_2', 'FOXP3 - regulatory T cells:Cyc_2_ch_3',
'CD8 - cytotoxic T cells:Cyc_3_ch_2', 'p53 - tumor suppressor:Cyc_3_ch_3',
'GATA3 - Th2 helper T cells:Cyc_3_ch_4', 'CD45 - hematopoietic cells:Cyc_4_ch_2',
'T-bet - Th1 cells:Cyc_4_ch_3', 'beta-catenin - Wnt signaling:Cyc_4_ch_4',
'HLA-DR - MHC-II:Cyc_5_ch_2', 'PD-L1 - checkpoint:Cyc_5_ch_3',
'Ki67 - proliferation:Cyc_5_ch_4', 'CD45RA - naive T cells:Cyc_6_ch_2',
'CD4 - T helper cells:Cyc_6_ch_3', 'CD21 - DCs:Cyc_6_ch_4', 'MUC-1 - epithelia:Cyc_7_ch_2',
'CD30 - costimulator:Cyc_7_ch_3', 'CD2 - T cells:Cyc_7_ch_4', 'Vimentin - cytoplasm:Cyc_8_ch_2',
'CD20 - B cells:Cyc_8_ch_3', 'LAG-3 - checkpoint:Cyc_8_ch_4', 'Na-K-ATPase - membranes:Cyc_9_ch_2',
'CD5 - T cells:Cyc_9_ch_3', 'IDO-1 - metabolism:Cyc_9_ch_4', 'Cytokeratin - epithelia:Cyc_10_ch_2',
'CD11b - macrophages:Cyc_10_ch_3', 'CD56 - NK cells:Cyc_10_ch_4', 'aSMA - smooth muscle:Cyc_11_ch_2',
'BCL-2 - apoptosis:Cyc_11_ch_3', 'CD25 - IL-2 Ra:Cyc_11_ch_4', 'CD11c - DCs:Cyc_12_ch_3',
'PD-1 - checkpoint:Cyc_12_ch_4', 'Granzyme B - cytotoxicity:Cyc_13_ch_2',
'EGFR - signaling:Cyc_13_ch_3', 'VISTA - costimulator:Cyc_13_ch_4',
'CD15 - granulocytes:Cyc_14_ch_2', 'ICOS - costimulator:Cyc_14_ch_4',
'Synaptophysin - neuroendocrine:Cyc_15_ch_3', 'GFAP - nerves:Cyc_16_ch_2',
'CD7 - T cells:Cyc_16_ch_3', 'CD3 - T cells:Cyc_16_ch_4', 'Chromogranin A - neuroendocrine:Cyc_17_ch_2',
'CD163 - macrophages:Cyc_17_ch_3', 'CD45RO - memory cells:Cyc_18_ch_3', 'CD68 - macrophages:Cyc_18_ch_4',
'CD31 - vasculature:Cyc_19_ch_3', 'Podoplanin - lymphatics:Cyc_19_ch_4', 'CD34 - vasculature:Cyc_20_ch_3',
'CD38 - multifunctional:Cyc_20_ch_4', 'CD138 - plasma cells:Cyc_21_ch_3', 'KNN', 'GraphID', 'CellId', 'X:X', 'Y:Y', 'Z:Z' ,'size:size']

In [9]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch
import numpy as np

# Identify numerical columns for features, excluding identifiers and potential non-feature columns
include_cols = icols

# Convert each row of the DataFrame into a Data object
data_list = []
for index, row in df.iterrows():
    # Features (x): Convert selected numerical columns to a float tensor
    x = torch.tensor(row[include_cols].values.astype(np.float32)).unsqueeze(0) # unsqueeze(0) to make it [1, num_features]


    data = Data(x=x, y=y)
    data_list.append(data)

# Initialize the DataLoader with the list of Data objects
loader = DataLoader(data_list, batch_size=32)

## Model
